In [1]:
pip install keras

     |████████████████████████████████| 1.3 MB 2.1 MB/s 
Note: you may need to restart the kernel to use updated packages.


# Initializing Libraries

In [2]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from PIL import Image

%matplotlib inline
from tensorflow import keras 
from tensorflow.keras.applications import ResNet50,ResNet101
import cv2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

2021-09-13 05:46:42.571354: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Getting the Data

In [4]:
train = pd.read_csv('/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')
train.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


# Another dataset

In [5]:
# As we are going to divide dataset
df = train.copy()

In [6]:
# Increasing the size of dataset without disturbing their corresponding ratios 

Misce = train[train["Class"]=='Miscellaneous']
Chris_tree = train[train["Class"]=='Christmas_Tree']
Jacket = train[train["Class"]=='Jacket']
Candle = train[train["Class"]=='Candle']
Airplane = train[train["Class"]=='Airplane']
Snowman = train[train["Class"]=='Snowman']

df = pd.concat([df,Misce])
df = pd.concat([df,Chris_tree])
df = pd.concat([df,Jacket])
df = pd.concat([df,Candle])
df = pd.concat([df,Airplane])
df = pd.concat([df,Snowman])

# Setting Path For Images Folder

In [7]:
TRAIN_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train'

# Model

In [8]:
def get_model(IMG_SIZE):
    base_model =applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    add_model = Sequential()
    add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    add_model.add(Dropout(0.3))
    add_model.add(Dense(64, activation='relu'))
    add_model.add(Dropout(0.4))

    add_model.add(Dense(6, activation='softmax'))

    model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    return model
#     model.summary()

# Preparation for kfolds

In [9]:
# Storing the average of all predictions

main_pred = []
error = []
data_kfold = pd.DataFrame()

In [10]:
# Creating X, Y for training 

train_y = df.Class
train_x = df.drop(['Class'],axis=1)

**I took 7 splits as we have 6 labels and even for worst case at least 1 label will have 2 occurence**

Increase EPOCHS variable if you are going for competition

In [11]:
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 1
N_SPLIT = 7

# Training And Predition

In [12]:
#Initializing Data Generators
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

# k-fold
kfold = StratifiedKFold(n_splits=N_SPLIT,shuffle=True,random_state=42)

# Variable for keeping count of split we are executing
j = 0

# K-fold Train and test for each split
for train_idx, val_idx in list(kfold.split(train_x,train_y)):
    x_train_df = df.iloc[train_idx]
    x_valid_df = df.iloc[val_idx]
    j+=1


    training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
    validation_set = validation_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
    model_test = get_model(IMG_SIZE)
    
    
    history = model_test.fit_generator( training_set,
                                        validation_data=validation_set,
                                        epochs = EPOCHS,
                                        steps_per_epoch=x_train_df.shape[0] // BATCH_SIZE
                                        )
    
    test_generator = ImageDataGenerator(rescale = 1./255)
    
    test_set = test_generator.flow_from_dataframe(dataframe=train, directory=TRAIN_PATH,
                                                 x_col="Image",y_col=None,
                                                 class_mode=None,
                                                 target_size=(IMG_SIZE,IMG_SIZE))
    
    pred= model_test.predict_generator(test_set, len(train) // BATCH_SIZE)
    predicted_class_indices=np.argmax(pred,axis=1)
                                       
    data_kfold[j] = predicted_class_indices
    gc.collect()

Found 11089 validated image filenames belonging to 6 classes.
Found 1849 validated image filenames belonging to 6 classes.


2021-09-13 05:47:00.657834: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-13 05:47:00.661660: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-13 05:47:00.702229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 05:47:00.702859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-13 05:47:00.702921: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-13 05:47:00.755700: I tensorflow/stream_executor/platform/def

94773248/94765736 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-13 05:47:05.403557: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-13 05:47:05.410041: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000175000 Hz
2021-09-13 05:47:10.082111: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-13 05:47:11.083021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-13 05:47:11.112503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


693/693 [==============================] - 93s 118ms/step - loss: 1.3770 - accuracy: 0.5539 - val_loss: 1.0581 - val_accuracy: 0.6128
Found 6469 validated image filenames.


...

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Found 11090 validated image filenames belonging to 6 classes.
Found 1848 validated image filenames belonging to 6 classes.


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


693/693 [==============================] - 75s 103ms/step - loss: 1.4278 - accuracy: 0.5501 - val_loss: 1.3465 - val_accuracy: 0.5319
Found 6469 validated image filenames.


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [13]:
# holder
data_kfold

,1,2,3,4,5,6,7
0,5,2,4,4,4,4,2
1,4,4,4,4,4,4,4
2,4,4,4,4,4,3,1
3,4,4,4,4,3,3,4
4,4,4,4,4,4,1,2
...,...,...,...,...,...,...,...
6464,4,4,4,4,4,4,4
6465,2,4,2,4,4,4,1
6466,4,4,4,2,4,2,4
6467,5,4,4,4,4,4,1


In [14]:
# testing on single iteration of HoldOut
predicted_class_indices = data_kfold[1]
labels=(training_set.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]

In [15]:
print("Accuracy of HandOut Method: ",accuracy_score(predictions,train.Class))

Accuracy of HandOut Method:  0.32168805070335443


In [16]:
ans = train.copy()

In [17]:
# Taking The Label with Maximum Occurences

labels=(training_set.class_indices)
labels2=dict((v,k) for k,v in labels.items())
import collections 
for i in range(len(data_kfold)):
    co = collections.Counter(data_kfold.loc[i])
    co = sorted(co.items(),key=lambda x: x[1],reverse=True)
    ans.Class.loc[i] = labels2[co[0][0]]

In [18]:
# Averaged K-Fold Output
print("Accuracy of K-Fold Method: ",accuracy_score(train.Class,ans.Class))

Accuracy of K-Fold Method:  0.4274230947596228


## **Averaged Accuracy of K-Fold > Accuracy by HoldOut**

# Please Upvote if it helped YOU ! 👍✌